In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [7]:
data = pd.read_csv('2003_01_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - Consumo de Cimento (t)
0,2003,620.268108,8.297178,0.745147,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,26.264888,19.479267,69.872005,1.873053,4.031463,6.084564e+07,2.670588e+06,16.831332,5.118946e+07,668.575000
1,2004,624.909873,8.226849,0.747078,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,26.348354,19.466165,69.446913,1.880115,3.935247,6.183013e+07,2.695788e+06,16.886950,5.150965e+07,716.682000
2,2005,653.029381,8.206660,0.748800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,26.412450,19.549355,69.408540,1.887298,3.737271,6.268667e+07,2.708770e+06,16.898198,5.167439e+07,737.634000
3,2006,727.816146,8.120431,0.750318,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,26.376895,19.468049,69.355795,1.848041,3.700564,6.354595e+07,2.735091e+06,16.937332,5.189736e+07,701.548000
4,2007,812.735644,8.093287,0.751639,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,26.405369,19.238651,68.805261,1.633719,3.889725,6.452787e+07,2.767528e+06,17.057531,5.233307e+07,840.901000
5,2008,869.834453,8.063005,0.752766,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,26.585520,19.247754,68.306560,1.265376,4.140784,6.535602e+07,2.793553e+06,17.127903,5.260918e+07,994.957000
6,2009,949.083048,8.095019,0.753704,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,26.514656,18.393406,65.147890,1.139212,4.367145,6.593967e+07,2.815133e+06,17.131657,5.267158e+07,998.465000
7,2010,1038.662223,8.057848,0.754460,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,26.516966,17.684947,62.460338,1.378783,4.559665,6.789156e+07,2.861375e+06,17.294037,5.339572e+07,1099.340000
8,2011,1134.526356,8.047069,0.755039,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,26.594241,18.048644,63.561988,1.548197,4.705270,6.985811e+07,2.887266e+06,17.471861,5.415240e+07,1204.074000
9,2012,1291.161206,6.695389,0.755027,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,26.504045,18.419653,65.231377,1.625213,4.879101,7.159016e+07,2.905726e+06,17.591515,5.472408e+07,1381.539000


In [8]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
0,620.268108,8.297178,0.745147,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,1432.274199,26.264888,19.479267,69.872005,1.873053,4.031463,6.084564e+07,2.670588e+06,16.831332,5.118946e+07
1,624.909873,8.226849,0.747078,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,4448.028829,26.348354,19.466165,69.446913,1.880115,3.935247,6.183013e+07,2.695788e+06,16.886950,5.150965e+07
2,653.029381,8.206660,0.748800,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,5908.968830,26.412450,19.549355,69.408540,1.887298,3.737271,6.268667e+07,2.708770e+06,16.898198,5.167439e+07
3,727.816146,8.120431,0.750318,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,4369.618410,26.376895,19.468049,69.355795,1.848041,3.700564,6.354595e+07,2.735091e+06,16.937332,5.189736e+07
4,812.735644,8.093287,0.751639,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,2543.203078,26.405369,19.238651,68.805261,1.633719,3.889725,6.452787e+07,2.767528e+06,17.057531,5.233307e+07
5,869.834453,8.063005,0.752766,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,1421.360784,26.585520,19.247754,68.306560,1.265376,4.140784,6.535602e+07,2.793553e+06,17.127903,5.260918e+07
6,949.083048,8.095019,0.753704,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,971.624734,26.514656,18.393406,65.147890,1.139212,4.367145,6.593967e+07,2.815133e+06,17.131657,5.267158e+07
7,1038.662223,8.057848,0.754460,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,896.219335,26.516966,17.684947,62.460338,1.378783,4.559665,6.789156e+07,2.861375e+06,17.294037,5.339572e+07
8,1134.526356,8.047069,0.755039,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,916.320612,26.594241,18.048644,63.561988,1.548197,4.705270,6.985811e+07,2.887266e+06,17.471861,5.415240e+07
9,1291.161206,6.695389,0.755027,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,884.118586,26.504045,18.419653,65.231377,1.625213,4.879101,7.159016e+07,2.905726e+06,17.591515,5.472408e+07


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      716.682000
1      737.634000
2      701.548000
3      840.901000
4      994.957000
5      998.465000
6     1099.340000
7     1204.074000
8     1381.539000
9     1618.657000
10    1411.234429
11    1241.100343
12    1083.128229
13    1220.058000
14    1316.386000
15    1446.064000
16    1555.266000
17    1788.146000
18    1947.017000
19            NaN
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
0,-1.615431,1.138190,-2.280941,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,...,-0.171952,-1.112674,1.211103,1.336789,1.301383,-0.989366,-1.541487,-1.957611,-1.480072,-1.517184
1,-1.595757,0.996694,-1.754360,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,...,1.738021,-0.863740,1.188974,1.185442,1.336103,-1.152216,-1.368334,-1.613429,-1.336381,-1.341516
2,-1.476578,0.956074,-1.284701,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,...,2.663281,-0.672578,1.329484,1.171780,1.371413,-1.487302,-1.217685,-1.436115,-1.307322,-1.251137
3,-1.159607,0.782590,-0.870554,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,...,1.688362,-0.778620,1.192156,1.153001,1.178422,-1.549431,-1.066554,-1.076628,-1.206220,-1.128812
4,-0.799691,0.727978,-0.510512,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,...,0.531635,-0.693697,0.804695,0.956992,0.124799,-1.229266,-0.893853,-0.633609,-0.895683,-0.889772
5,-0.557687,0.667052,-0.203166,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,...,-0.178864,-0.156406,0.820071,0.779437,-1.686003,-0.804334,-0.748198,-0.278161,-0.713875,-0.738289
6,-0.221806,0.731462,0.052891,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,...,-0.463696,-0.367754,-0.622950,-0.345156,-2.306236,-0.421204,-0.645544,0.016575,-0.704176,-0.704053
7,0.157860,0.656677,0.259068,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,...,-0.511452,-0.360863,-1.819558,-1.302016,-1.128486,-0.095354,-0.302243,0.648153,-0.284662,-0.306777
8,0.564163,0.634991,0.416773,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,...,-0.498721,-0.130398,-1.205263,-0.909792,-0.295635,0.151091,0.043635,1.001767,0.174749,0.108355
9,1.228033,-2.084484,0.413719,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,...,-0.519116,-0.399402,-0.578617,-0.315432,0.082982,0.445311,0.348269,1.253888,0.483878,0.421994


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      716.682000
1      737.634000
2      701.548000
3      840.901000
4      994.957000
5      998.465000
6     1099.340000
7     1204.074000
8     1381.539000
9     1618.657000
10    1411.234429
11    1241.100343
12    1083.128229
13    1220.058000
14    1316.386000
15    1446.064000
16    1555.266000
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
17,1.231364,-1.557487,-0.237865,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.250215,2.664695,-1.918742,-2.3518,-0.291047,2.189261,1.031802,0.252569,1.048175,1.0512


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1788.146
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [14]:
train_input.iloc[-(len(train_input)//5):]

,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
14,0.468011,-0.966352,1.182926,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,...,-0.716779,0.270204,-0.301868,-0.298426,-0.237906,0.848163,1.227775,0.169850,1.129461,1.178019
15,0.586822,-1.251397,0.881624,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,...,-0.578390,1.473375,-0.808369,-1.041470,-0.399178,0.815250,1.258529,0.142121,1.173132,1.221186
16,0.888425,-1.395196,0.354581,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,...,-0.390895,3.225543,-1.778185,-2.376735,-0.386996,1.338967,1.204761,0.205554,1.138220,1.178775


In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [18]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2618637088, 1590501093, 3694428389, 235300612, 502978269, 1187900289, 1578312029, 175378524, 985273120, 1043437018, 3179316661, 3457310725, 690136942, 616159050, 1088569791, 3396253397, 808514084, 896035717, 1836452103, 1632990204, 467275695, 1014908263, 2658118811, 2965341161, 2778768137, 3398268538, 2142707451, 391533856, 16630433, 3350831349, 3597275413, 3157325219, 4068675682, 4052212137, 4090191144, 3297164120, 3433491314, 704017191, 1764513344, 1574313555, 1385950710, 1143178480, 200716854, 2838200012, 1273464489, 3362057100, 3422028919, 4283725312, 335710873, 3954051231, 1301299773, 2328826490, 2548795293, 4092790949, 1141097454, 1959306257, 2464452420, 1212547495, 2618790316, 265411508, 3814725419, 4260865994, 117001371, 3161805420, 615021764, 1355484670, 3337651477, 1109748105, 750805682, 2857088103, 2232792135, 1223838395, 4103307537, 3050668021, 2556468761, 4049012273, 2514602529, 1772118306, 3532935837, 4027372696, 757485159, 2176576342, 3679496924, 1616407097, 3627742045,

2023-10-04 14:38:47.739451: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 12249.3271484375


Step: 4 ___________________________________________
val_loss: 9536.6298828125
winner_seed: 502978269


Step: 5 ___________________________________________
val_loss: 13571.19140625


Step: 6 ___________________________________________
val_loss: 12827.796875


Step: 7 ___________________________________________
val_loss: 16663.861328125


Step: 8 ___________________________________________
val_loss: 12420.6416015625


Step: 9 ___________________________________________
val_loss: 13907.3701171875


Step: 10 ___________________________________________
val_loss: 12302.765625


Step: 11 ___________________________________________
val_loss: 13625.7939453125


Step: 12 ___________________________________________
val_loss: 9537.5166015625


Step: 13 ___________________________________________
val_loss: 19553.931640625


Step: 14 ___________________________________________
val_loss: 11428.3310546875


Step: 15 ___________________________________________
val_loss: 147

2023-10-04 14:43:16.579947: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:43:16.611671: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:43:16.643996: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:43:16.951780: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 14:43:16.987516: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 9729.00390625


Step: 18 ___________________________________________


...

val_loss: 11684.3837890625


Step: 81 ___________________________________________


2023-10-04 15:02:54.479601: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:02:54.516898: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-04 15:02:54.546696: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 10214.2822265625


Step: 82 ___________________________________________
val_loss: 6486.2734375
winner_seed: 3679496924


Step: 83 ___________________________________________
val_loss: 15153.6123046875


Step: 84 ___________________________________________
val_loss: 9580.1142578125


Step: 85 ___________________________________________
val_loss: 10149.6650390625


Step: 86 ___________________________________________
val_loss: 13667.8330078125


Step: 87 ___________________________________________
val_loss: 10500.8525390625


Step: 88 ___________________________________________
val_loss: 11175.9609375


Step: 89 ___________________________________________
val_loss: 10787.8076171875


Step: 90 ___________________________________________


2023-10-04 15:05:54.721827: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 17904.87890625


Step: 91 ___________________________________________
val_loss: 10523.1630859375


Step: 92 ___________________________________________
val_loss: 9533.90234375


Step: 93 ___________________________________________
val_loss: 15285.1767578125


Step: 94 ___________________________________________
val_loss: 10971.3876953125


Step: 95 ___________________________________________
val_loss: 12386.4560546875


Step: 96 ___________________________________________
val_loss: 9534.9677734375


Step: 97 ___________________________________________
val_loss: 9165.66015625


Step: 98 ___________________________________________
val_loss: 13635.6533203125


Step: 99 ___________________________________________
val_loss: 14019.58203125


final_seed: 3679496924


In [19]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 476ms/step - loss: 1403882.1250 - val_loss: 2012072.6250
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 1362677.6250 - val_loss: 913334.0000
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 1061556.0000 - val_loss: 2204708.2500
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 1397150.3750 - val_loss: 763470.8125
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 998128.5000 - val_loss: 698083.5000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 959750.9375 - val_loss: 665424.6875
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 965161.3125 - val_loss: 697251.6875
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 948452.0000 - val_loss: 642599.0000
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 928866.2500 - val_loss: 659007.2500
Epo

Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 297298.2188 - val_loss: 259366.0781
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 292322.3750 - val_loss: 258383.3125
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 287262.6562 - val_loss: 256659.3281
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 282535.7812 - val_loss: 252849.8750
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 278530.0938 - val_loss: 245087.8750
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 273331.8125 - val_loss: 247654.9531
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 268842.0000 - val_loss: 242865.9531
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 264606.1562 - val_loss: 237851.2969
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 260269.9375 - val_loss: 235276.2969
E

1/1 [==============================] - 0s 13ms/step - loss: 97699.9141 - val_loss: 108902.6016
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 96668.5078 - val_loss: 108747.4375
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 95359.4844 - val_loss: 107282.4141
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 94074.3906 - val_loss: 106178.3125
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 92529.6328 - val_loss: 104122.3516
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 91284.1484 - val_loss: 108291.7891
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 90421.3750 - val_loss: 101653.5625
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 89176.2500 - val_loss: 101954.4297
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 87736.3750 - val_loss: 104523.3750
Epoch 157/10000
1

1/1 [==============================] - 0s 13ms/step - loss: 42489.1133 - val_loss: 56038.6250
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 42096.5391 - val_loss: 55028.2617
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 41743.1562 - val_loss: 54722.2695
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 41385.7773 - val_loss: 54242.1445
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 41034.2656 - val_loss: 53813.7344
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 40688.5430 - val_loss: 53379.2539
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 40348.4531 - val_loss: 52953.3320
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 40013.9258 - val_loss: 52532.1133
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 39370.0039 - val_loss: 50625.5273
Epoch 231/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 26287.0156 - val_loss: 33218.9688
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 26465.5000 - val_loss: 33654.3203
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 26103.5547 - val_loss: 32777.6836
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 26055.8008 - val_loss: 32966.7031
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 26173.8574 - val_loss: 33201.3867
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 25750.0664 - val_loss: 32162.6621
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 25639.9883 - val_loss: 32202.4199
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 25535.9707 - val_loss: 31973.2598
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 25433.8926 - val_loss: 31814.8926
Epoch 305/10000
1/1 [=====

...

1/1 [==============================] - 0s 13ms/step - loss: 18482.9629 - val_loss: 15551.0752
Epoch 890/10000
1/1 [==============================] - 0s 14ms/step - loss: 18998.5840 - val_loss: 16369.3076
Epoch 891/10000
1/1 [==============================] - 0s 13ms/step - loss: 18152.1250 - val_loss: 15912.1240
Epoch 892/10000
1/1 [==============================] - 0s 13ms/step - loss: 18265.8594 - val_loss: 15733.1572
Epoch 893/10000
1/1 [==============================] - 0s 13ms/step - loss: 19013.1602 - val_loss: 15595.1094
Epoch 894/10000
1/1 [==============================] - 0s 13ms/step - loss: 18008.1836 - val_loss: 15794.6484
Epoch 895/10000
1/1 [==============================] - 0s 13ms/step - loss: 19438.8555 - val_loss: 15782.2529
Epoch 896/10000
1/1 [==============================] - 0s 13ms/step - loss: 17759.3457 - val_loss: 15442.7861
Epoch 897/10000
1/1 [==============================] - 0s 13ms/step - loss: 18813.3027 - val_loss: 16303.3428
Epoch 898/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 18238.1289 - val_loss: 16179.6357
Epoch 964/10000
1/1 [==============================] - 0s 13ms/step - loss: 18683.6504 - val_loss: 15044.4639
Epoch 965/10000
1/1 [==============================] - 0s 13ms/step - loss: 18726.8867 - val_loss: 15018.7217
Epoch 966/10000
1/1 [==============================] - 0s 13ms/step - loss: 17336.8516 - val_loss: 15809.0479
Epoch 967/10000
1/1 [==============================] - 0s 13ms/step - loss: 17754.9941 - val_loss: 14654.3076
Epoch 968/10000
1/1 [==============================] - 0s 13ms/step - loss: 18186.2949 - val_loss: 15354.1650
Epoch 969/10000
1/1 [==============================] - 0s 13ms/step - loss: 17878.2031 - val_loss: 15586.8828
Epoch 970/10000
1/1 [==============================] - 0s 13ms/step - loss: 18077.0918 - val_loss: 14831.7939
Epoch 971/10000
1/1 [==============================] - 0s 13ms/step - loss: 18027.7988 - val_loss: 15423.1904
Epoch 972/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 18900.9941 - val_loss: 15918.7939
Epoch 1038/10000
1/1 [==============================] - 0s 13ms/step - loss: 18890.4023 - val_loss: 15659.8682
Epoch 1039/10000
1/1 [==============================] - 0s 13ms/step - loss: 19000.2324 - val_loss: 15418.1514
Epoch 1040/10000
1/1 [==============================] - 0s 13ms/step - loss: 18901.2578 - val_loss: 15806.5146
Epoch 1041/10000
1/1 [==============================] - 0s 13ms/step - loss: 18899.4121 - val_loss: 15915.5156
Epoch 1042/10000
1/1 [==============================] - 0s 13ms/step - loss: 18898.3672 - val_loss: 15898.3467
Epoch 1043/10000
1/1 [==============================] - 0s 13ms/step - loss: 18897.4902 - val_loss: 15900.5283
Epoch 1044/10000
1/1 [==============================] - 0s 13ms/step - loss: 18888.6641 - val_loss: 15826.0977
Epoch 1045/10000
1/1 [==============================] - 0s 13ms/step - loss: 18920.2246 - val_loss: 15655.5312
Epoch 1046/10000
1

Epoch 1111/10000
1/1 [==============================] - 0s 13ms/step - loss: 18947.1660 - val_loss: 16131.7158
Epoch 1112/10000
1/1 [==============================] - 0s 13ms/step - loss: 18914.4395 - val_loss: 16006.6670
Epoch 1113/10000
1/1 [==============================] - 0s 13ms/step - loss: 18989.8945 - val_loss: 16277.2148
Epoch 1114/10000
1/1 [==============================] - 0s 13ms/step - loss: 18892.9629 - val_loss: 15822.1094
Epoch 1115/10000
1/1 [==============================] - 0s 13ms/step - loss: 19062.8633 - val_loss: 16397.6172
Epoch 1116/10000
1/1 [==============================] - 0s 13ms/step - loss: 18892.3320 - val_loss: 15794.2227
Epoch 1117/10000
1/1 [==============================] - 0s 13ms/step - loss: 18887.1348 - val_loss: 15886.2568
Epoch 1118/10000
1/1 [==============================] - 0s 13ms/step - loss: 18939.7031 - val_loss: 15854.5400
Epoch 1119/10000
1/1 [==============================] - 0s 13ms/step - loss: 18893.5684 - val_loss: 15810.6523
E

Epoch 1185/10000
1/1 [==============================] - 0s 13ms/step - loss: 19064.1270 - val_loss: 16355.6299
Epoch 1186/10000
1/1 [==============================] - 0s 13ms/step - loss: 18925.0645 - val_loss: 16033.6221
Epoch 1187/10000
1/1 [==============================] - 0s 13ms/step - loss: 18922.1719 - val_loss: 16054.1094
Epoch 1188/10000
1/1 [==============================] - 0s 13ms/step - loss: 18921.7793 - val_loss: 16046.9541
Epoch 1189/10000
1/1 [==============================] - 0s 13ms/step - loss: 18921.2480 - val_loss: 16046.3838
Epoch 1190/10000
1/1 [==============================] - 0s 13ms/step - loss: 18920.3379 - val_loss: 16069.8311
Epoch 1191/10000
1/1 [==============================] - 0s 12ms/step - loss: 18920.1016 - val_loss: 16040.4541
Epoch 1192/10000
1/1 [==============================] - 0s 13ms/step - loss: 18919.2656 - val_loss: 16065.5869
Epoch 1193/10000
1/1 [==============================] - 0s 13ms/step - loss: 18962.3203 - val_loss: 15933.1396
E

In [20]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[1788.146](test_target) - [[1422.7236]](prediction) = 365.42236718749996


In [21]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [22]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
0,-1.135241,1.856064,-1.736946,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,...,-0.735605,-1.668859,0.658811,0.818469,0.897872,-0.049168,-1.491648,-1.400684,-1.277688,-1.429098
1,-1.103644,0.998090,-1.109149,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,...,0.949295,-0.815603,0.637928,0.646365,0.922275,-0.392711,-1.037323,-0.987319,-0.902262,-0.957641
2,-0.912233,0.751791,-0.549213,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,...,1.765522,-0.160368,0.770529,0.630829,0.947093,-1.099597,-0.642046,-0.774364,-0.826338,-0.715083
3,-0.403156,-0.300147,-0.055461,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,...,0.905488,-0.523843,0.640931,0.609475,0.811448,-1.230663,-0.245503,-0.342617,-0.562184,-0.386786
4,0.174895,-0.631291,0.373786,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,...,-0.114930,-0.232754,0.275279,0.386583,0.070900,-0.555252,0.207634,0.189451,0.249166,0.254748
5,0.563570,-1.000717,0.740209,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,...,-0.741702,1.608887,0.289790,0.184677,-1.201836,0.341171,0.589810,0.616348,0.724182,0.661296
6,1.103019,-0.610161,1.045484,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,...,-0.992970,0.884461,-1.072007,-1.094153,-1.637772,1.149409,0.859156,0.970328,0.749522,0.753178
7,1.712789,-1.063629,1.291291,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,...,-1.035098,0.908079,-2.201261,-2.182245,-0.809981,1.836811,1.759919,1.728857,1.845602,1.819387


train_target:


,Mato Grosso - Consumo de Cimento (t)
0,716.682
1,737.634
2,701.548
3,840.901
4,994.957
5,998.465
6,1099.340
7,1204.074


test_input:


...

,Mato Grosso - Produção de Cimento (t),Mato Grosso - Desemprego,Mato Grosso - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso - Valor Cimento Portland,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado
17,1.231364,-1.557487,-0.237865,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,...,-0.250215,2.664695,-1.918742,-2.3518,-0.291047,2.189261,1.031802,0.252569,1.048175,1.0512


test_target:


,Mato Grosso - Consumo de Cimento (t)
17,1788.146


1/1 [==============================] - 0s 35ms/step
Error: 365.42236718749996




[1104.06103515625,
 1212.5408935546875,
 1344.60009765625,
 1381.9927978515625,
 1407.1729736328125,
 1296.547119140625,
 1329.3558349609375,
 1298.92724609375,
 1327.1231689453125,
 1422.7236328125]

In [23]:
display(targets)
display(predictions)

[1381.539,
 1618.657,
 1411.2344285714282,
 1241.1003428571426,
 1083.1282285714285,
 1220.058,
 1316.386,
 1446.064,
 1555.266,
 1788.146]

[1104.06103515625,
 1212.5408935546875,
 1344.60009765625,
 1381.9927978515625,
 1407.1729736328125,
 1296.547119140625,
 1329.3558349609375,
 1298.92724609375,
 1327.1231689453125,
 1422.7236328125]

In [24]:
mae = mean_absolute_error(predictions, targets)
mae

204.53265085100446

In [25]:
porcentage = mae/np.mean(targets)
porcentage

0.14545496693579324